In [5]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from scipy.linalg import toeplitz
from numpy.linalg import inv
from sklearn.metrics import mean_squared_error

In [10]:
noisy_12 = loadmat("data/My_noisy_H_12.mat")['My_noisy_H']
noisy_22 = loadmat("data/My_noisy_H_22.mat")['My_noisy_H']
perfect = loadmat("data/Perfect_H_40000.mat")['My_perfect_H']

In [77]:
def mmse_channel_estimation(H_pilot, X_pilot, noise_var, num_pilots):
    R_hh = toeplitz(np.exp(-0.1 * np.arange(num_pilots)))  # Ma trận tương quan
    R_yh = R_hh[:, :len(X_pilot)]  # Ma trận tương quan kênh với pilot
    R_yy = R_yh.T @ R_hh + noise_var * np.eye(len(X_pilot)) # Ma trận tương quan của tín hiệu nhận được

    # Ước lượng MMSE
    H_mmse = R_yh @ inv(R_yy) @ H_pilot
    return H_mmse

In [26]:
n_subcarriers = 72
num_pilots = 24
num_timeslots = 14
SNR_dB = np.arange(0, 30, 5)
mse_mmse = []
mse_deep = []

In [63]:
snr = 0
noise_var = 10 ** (-snr / 10)

# Pilot và kênh truyền tại vị trí pilot
pilot_idx = np.linspace(0, n_subcarriers - 1, num_pilots).astype(int)
X_pilot = np.ones(num_pilots)  # Pilot giả sử là tín hiệu 1
H_pilot = perfect[:, pilot_idx] \
    + np.sqrt(noise_var / 2) * (np.random.randn(perfect.shape[0], num_pilots, num_timeslots) \
                                + 1j * np.random.randn(perfect.shape[0], num_pilots, num_timeslots))

# Ước lượng kênh bằng MMSE
H_mmse = mmse_channel_estimation(H_pilot[0], X_pilot, noise_var, num_pilots)

In [79]:
H_mmse.shape

(40000, 24, 14)

In [73]:
print(R_hh.shape)
print(R_yh.shape)
print((R_yh.T @ R_hh).shape)

(24, 24)
(24, 24)
(24, 24)


In [61]:
print(H_pilot[0].shape)
print(H_pilot[0, :, 0].shape)
print(X_pilot.shape)

(24, 14)
(24,)
(24,)


In [32]:
print(perfect[:, pilot_idx].shape)
print(np.random.randn(perfect.shape[0], num_pilots, num_timeslots).shape)

(40000, 24, 14)
(40000, 24, 14)
